In [3]:
import xlwt, xlrd

path_origin_post = '原数据/posts.xlsx' # 我勒个操，用xls就报UnicodeError异常，改成xlsx就好了，我勒个操操操操
path_origin_comment = '原数据/comments.xls'

path_bad_post = '坏链接/posts_bad.xls'
path_bad_comment = '坏链接/comments_bad.xls'

path_rest_post = '缺链接/posts_rest.xls'
path_rest_comment = '缺链接/comments_rest.xls'

workbook_origin_comment = xlrd.open_workbook(path_origin_comment, encoding_override="gbk")
sheet_origin_comment = workbook_origin_comment.sheet_by_index(0)
origin_comments = [[] for i in range(sheet_origin_comment.nrows)]
for i in range(sheet_origin_comment.nrows):
#     print(sheet_origin_comment.row_values(i))
    idx, cnt, text = sheet_origin_comment.row_values(i)
    origin_comments[int(idx)-1].append((int(cnt), text))

origin_post_useful = []
origin_comment_useful = []
workbook_origin_post = xlrd.open_workbook(path_origin_post, encoding_override="gbk")
sheet_origin_post = workbook_origin_post.sheet_by_index(0)
for i in range(sheet_origin_post.nrows):
    if sheet_origin_post.row_values(i)[1] != '':
        origin_post_useful.append(sheet_origin_post.row_values(i)[1])
        origin_comment_useful.append(origin_comments[i])

workbook_bad_comment = xlrd.open_workbook(path_bad_comment, encoding_override="gbk")
sheet_bad_comment = workbook_bad_comment.sheet_by_index(0)
bad_comment = [[] for i in range(sheet_bad_comment.nrows)]
for i in range(sheet_bad_comment.nrows):
#     print(sheet_origin_comment.row_values(i))
    idx, cnt, text = sheet_bad_comment.row_values(i)
    bad_comment[int(idx)-1].append((int(cnt), text))    
        
workbook_bad_post = xlrd.open_workbook(path_bad_post, encoding_override="gbk")
sheet_bad_post = workbook_bad_post.sheet_by_index(0)
bad_post = []
for i in range(sheet_bad_post.nrows):
    idx, date, text, link = sheet_bad_post.row_values(i)
    bad_post.append(text)

workbook_rest_comment = xlrd.open_workbook(path_rest_comment, encoding_override="gbk")
sheet_rest_comment = workbook_rest_comment.sheet_by_index(0)
rest_comment = [[] for i in range(sheet_rest_comment.nrows)]
for i in range(sheet_rest_comment.nrows):
#     print(sheet_origin_comment.row_values(i))
    idx, cnt, text = sheet_rest_comment.row_values(i)
    rest_comment[int(idx)-1].append((int(cnt), text))    
        
workbook_rest_post = xlrd.open_workbook(path_rest_post, encoding_override="gbk")
sheet_rest_post = workbook_rest_post.sheet_by_index(0)
rest_post = []
for i in range(sheet_rest_post.nrows):
    idx, date, text, link = sheet_rest_post.row_values(i)
    rest_post.append(text)

In [4]:
all_post = []
all_comment = []

num  = 0
for idx, value in enumerate(origin_post_useful):
    num += 1
    all_post.append((num, value))
    for value in origin_comment_useful[idx]:
        all_comment.append((num, ) + value)

for idx, value in enumerate(bad_post):
    num += 1
    all_post.append((num, value))
    for value in bad_comment[idx]:
        all_comment.append((num, ) + value)

for idx, value in enumerate(rest_post):
    num += 1
    all_post.append((num, value))
    for value in rest_comment[idx]:
        all_comment.append((num, ) + value)

print(len(all_post), len(all_comment), sep=' ')

323 11882


In [56]:
# 整理写入的是不加reaction，comment, share 的信息
w = xlwt.Workbook()
sheet = w.add_sheet('post')
for idx, value in enumerate(all_post):
    sheet.write(idx, 0, value[0])
    sheet.write(idx, 1, value[1])
w.save('post.xls')

w = xlwt.Workbook()
sheet = w.add_sheet('comment')
for idx, value in enumerate(all_comment):
    sheet.write(idx, 0, value[0])
    sheet.write(idx, 1, value[1])
    sheet.write(idx, 2, value[2])
w.save('comment.xls')


323 11882


In [2]:
from bs4 import BeautifulSoup
import re

re_commentlink      = '<a(?:.[^>]*?)href="(.[^"]*?)"(?:[^>]*?)>(?:[0-9]* Comments|1 Comment|Comment)</a>'
pattern_commentlink = re.compile(re_commentlink)

with open('posts.html', 'rb') as f:
    a = f.read()
soup = BeautifulSoup(a.decode('utf-8'), 'lxml')

In [ ]:
divs = soup.find_all('div', attrs={'data-sigil': "reactions-bling-bar"})
articles = soup.find_all('article')
for idx, item in enumerate(articles):
    footer = item.find('footer')
    if footer:
        cnt_link = str(pattern_commentlink.findall(str(item))[0])
        content = footer.find('div', attrs={'data-sigil':"reactions-bling-bar"})
        print("-"*30)
        print(cnt_link)
        num_reaction = num_share = num_comment = 0
        if content:
            reaction = content.find('div', attrs={"data-sigil":"reactions-sentence-container"})
            if reaction:
                num_reaction = str(reaction.text)
                num_reaction = re.sub('\s+', ' ', num_reaction)
            comment = re.findall(r'([0-9]+)\s[Comment|Comments]', str(content))
            if comment:
                num_comment = comment[0]
            share = re.findall(r'([0-9]+)\s[Share|Shares]', str(content))
            if share:
                num_share = share[0]
        link = 'https://m.facebook.com' + cnt_link[:cnt_link.find('?')+1]
        for all_post